In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')
df.head()

In [ ]:
df.shape

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)), x['measurable_impressions'])*1000 , axis=1)
df.drop(columns=['total_revenue'], inplace=True)
df = df[df['CPM'] >= 0]
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df_train = df[df['date'] < '2019-06-22']
df_test = df[df['date'] >= '2019-06-22']
test = df_test['CPM'].quantile(0.95)

In [ ]:
test_data = df_test[df_test['CPM'] < test ]
train_data = df_train[df_train['CPM'] < test]

In [ ]:
train_data.shape, test_data.shape

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
X = train_data.drop(columns=['date', 'CPM'])
y = train_data['CPM']

X_test = test_data.drop(columns=['date', 'CPM'])
y_test = test_data['CPM']

In [ ]:
model = lgb.LGBMRegressor()

model.fit(X, y)

In [ ]:
y_pred = model.predict(X_test)
MSE(y_test, y_pred)